In [3]:
import numpy as np
import pandas as pd
import time
import os
import math
import random
from numba import jit
import gc

In [4]:
gc.enable()
root_dir = './'
doc_dir = './Document/'
query_dir = './Query/'

In [5]:
doc_names = []
query_names = []

with open('doc_list.txt', 'r') as f:
    doc_names = [line.strip() for line in f]

with open('query_list.txt', 'r') as f:
    query_names = [line.strip() for line in f]


In [6]:
training_set = {}

for i in range(len(doc_names)):
    with open(doc_dir + doc_names[i] , 'r') as f:
        next(f)
        next(f)
        next(f)
        words = []
        for line in f:
            words+=line.rstrip()[:-3].split(' ')
            
        training_set[i]=words

In [7]:
query_set = {}
for i in range(len(query_names)):
    with open(query_dir + query_names[i] , 'r') as f:
        words = []
        for line in f:
             words+=line.rstrip()[:-3].split(' ')
        query_set[i] = words

In [8]:
len(query_set)
len(query_set[0])

210

In [9]:
tf_dj_wi = {}
idf_wi = {}
vc_dj = {}
word_list = {}

In [10]:
#tf
for j in range(len(training_set)):
    tf_dj_wi[j] = {}
    vc_dj[j] = vc = pd.value_counts(training_set[j])
    
    vc_index = list(vc.index)
    for i in range(len(vc_index)):
        tf_dj_wi[j][vc_index[i]] = vc[i] / len(vc)

In [11]:
#word_list
word_list = {}
all_index = []
for j in range(len(vc_dj)):
    all_index += list(vc_dj[j].index)
    
w = np.unique(all_index)
for i in range(len(w)):
    word_list[str(w[i])] = i

In [12]:
len(word_list)

13290

In [13]:
#idf
idf_wi = {}
for j in range(len(training_set)):
    vc = vc_dj[j]
    vc_index = list(vc.index)
    
    for i in range(len(vc)):
        if vc_index[i] not in idf_wi:
            idf_wi[vc_index[i]] = vc[i]
        else:
            idf_wi[vc_index[i]] = idf_wi[vc_index[i]] + vc[i]

len_docs = len(training_set)
for word in idf_wi:
    idf_wi[word] = math.log10(len_docs/idf_wi[word])

In [14]:
idf_wi['516']

2.400825696909526

In [15]:
#tf(query_set)
def calc_tf_q(query_set):
    new_tf_qj_wi = {}
    for j in range(len(query_set)):
        new_tf_qj_wi[j] = {}
        query_vc = pd.value_counts(query_set[j])
        vc_index = list(query_vc.index)
        
        for i in range(len(vc_index)):
            new_tf_qj_wi[j][vc_index[i]] = query_vc[i]

    return new_tf_qj_wi



In [16]:
%%time
tf_qj_wi = calc_tf_q(query_set)

CPU times: user 3.04 s, sys: 43.8 ms, total: 3.08 s
Wall time: 3.15 s


In [17]:
def expand(arr):
    d = [0]*len(word_list)
    wlist = list(word_list)
    
    for word in arr:
        d[word_list[word]] = arr[word] 

    return d

In [18]:
vectors = {}
norms = {}
def vector_norm(tf):
    s_tf = str(tf)
    if s_tf not in vectors and s_tf not in norms:
        vectors[s_tf] = tf2vector(tf)
        norms[s_tf] = math.sqrt(sum(np.power(vectors[s_tf],2)))
        return vectors[s_tf], norms[s_tf]
    else:
        return vectors[s_tf], norms[s_tf]
    
    

In [19]:
from sklearn.metrics.pairwise import cosine_similarity 

In [26]:
tfidf_dj = [[0]*len(word_list)]*len(training_set)
for j in range(len(training_set)):
    tfidf_dj[j] = tf2vector(tf_dj_wi[j])

In [25]:
#clac similarity of query k
def sim(qk):

    vector_q,norm_q = vector_norm(tf_qj_wi[qk])
    sim_dj = cosine_similarity(vector_q, tfidf_dj)

    return sortDocs(sim_dj)

In [24]:
def rele_non_rele(arr):
    rele = arr[0:3]
    non_rele = arr[len(arr)-50:len(arr)-1]
    return rele, non_rele

In [23]:
#sort documents
def sortDocs(sim_dj):
    ls = [ doc_names.index(name) for _,name in sorted(zip(sim_dj,doc_names))]
    print(len(ls))
    return reversed(ls)

In [22]:
def tf2vector(tf):
    vector = [0]*len(word_list)
    for word in tf:
        if word in  word_list: #some word might not in word list of documents
            #vector = tf*idf 
            vector[word_list[word]] = tf[word]*idf_wi[word] 
    return vector

In [ ]:
# calc new query
def new_query(qi):
#     print('new query',time.ctime())
    
    alpha = 0.3
    beta = 1
    gamma = 0

    vector_q = tf2vector(tf_qj_wi[qi])
    
#     print('start sim',time.ctime())
    rel, non_rel = rele_non_rele(sim(qi))
#     print('rel_non_rel sim done',time.ctime())

    
    
    sum_rel = [0]*len(word_list)
    sum_non_rel = [0]*len(word_list)
    
    
    #calc rel, non_rel together
#     print('start combine rel non_rel',time.ctime())

    for j in range(len(rel)):
        rel_vector_d = tf2vector(tf_dj_wi[rel[j]])
    for j in range(len(non_rel)):
        non_rel_vector_d = tf2vector(tf_dj_wi[non_rel[j]])
        
        sum_rel = np.add(sum_rel,rel_vector_d)
        sum_non_rel = np.add(sum_non_rel, non_rel_vector_d)
        
#     print('combine rel non_rel done',time.ctime())

    new_q = np.multiply(vector_q, alpha) + np.multiply(sum_rel, beta) - np.multiply(sum_non_rel, gamma/50)
#     print('complete parameters setting', time.ctime())
    return new_q

In [28]:
a1 = [2,1,3,4]
a2 = [11,0,12,13]
# print(np.add(a1,a2)[3])
# np.power(a2,2)
# str(a1)
print(cosine_similarity(a1, a2))

dot = np.dot(a1,a2)
norm1 = math.sqrt(sum([math.pow(a,2) for a in a1]))
norm2 = math.sqrt(sum([math.pow(a,2) for a in a2]))
sim = dot/ norm1/ norm2
print(sim)


[[0.90369611]]
0.9036961141150639


/Users/Dennis/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Dennis/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [244]:
# calc new query

# print submission title

time_str = time.ctime()
with open(time_str+'submission.txt', 'w') as t:
    t.write('Query,RetrievedDocuments')

for i in range(len(query_set)):
    
    with open(time_str+'submission.txt', 'a') as t:
        t.write('\n')

    new_q = new_query(i)
    
    #array to dictionary
    tf_q = {}
    wl = list(word_list)
    for w in range(len(wl)):
        tf_q[wl[w]] = new_q[w]
    
    print(time.ctime(),'done q=',i)
    
    #sim
    vector_q,norm_q = vector_norm(tf_q)
    sim_dj = cosine_similarity(vector_q,tfidf_dj)
    

    docs = [name[i] for i in sortDocs(sim_dj)]
    
    with open(time_str+'submission.txt', 'a') as t:
        t.write('%s,' % query_names[i])

        for doc in docs:
            t.write('%s ' % doc)

    

/Users/Dennis/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


1


TypeError: 'list_reverseiterator' object is not subscriptable

In [31]:
a1 = [2,1,3,4]
a2 = [11,0,12,13]

[x for _,x in reversed(sorted(zip(a1,a2)))]

[13, 12, 11, 0]